In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# 모델 정의
class MyResNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyResNet, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.linear(x)
        return x

# 모델 인스턴스화
input_size = 10
output_size = 5
model = MyResNet(input_size, output_size)

# 예제 입력 데이터 생성
input_data = torch.randn((3, input_size))

# 모델에 입력 전달하여 출력 얻기
output_data = model(input_data)

print("모델 아키텍처:\n", model)
print("모델 출력:\n", output_data)


모델 아키텍처:
 MyResNet(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)
모델 출력:
 tensor([[ 0.2323, -0.1574, -0.5751, -0.1335,  0.6668],
        [-0.2477, -0.3730, -0.6910, -0.1203,  0.4183],
        [-0.0042, -0.4317, -0.3015, -0.9999,  0.9447]],
       grad_fn=<AddmmBackward0>)
